In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path
import pandas as pd

In [4]:
df_checks = pd.read_csv('https://raw.githubusercontent.com/amiapmorais/rpgai/main/DnDChecks.csv')

In [5]:
# Adicionando Travel e Explore que percebi que ficou faltando
df_extra = pd.DataFrame([['Travel', 'Survival'], ['Explore', 'Survival'], ['Investigate', 'Investigation']], columns=['text_skill', '5e_skill'])
df_checks = df_checks.append(df_extra, ignore_index=True)

In [ ]:
df_checks.head()

,text_skill,5e_skill
0,Climb,Athletics
1,Balance,Acrobatics
2,Sleight of Hand,Sleight of Hand
3,Hide,Stealth
4,Knowledge (arcana),Arcana


In [6]:
# fonte do código: https://stackoverflow.com/questions/8153823/how-to-fix-this-attributeerror
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        # initialize the base class
        HTMLParser.__init__(self)

    def read(self, data):
        # clear the current output before re-use
        self._lines = []
        # re-set the parser's state before re-use
        self.reset()
        self.feed(data)
        return ''.join(self._lines)

    def handle_data(self, d):
        self._lines.append(d)

def strip_tags(html):
    s = MLStripper()
    return s.read(html)

Vale ressaltar que alguns registros dos arquivos csv são htmls de páginas 404, logo não possuem dado algum. Como o exemplo abaixo.

> Indented block



In [ ]:
#df['list1_link'][1]

'https://www.tavern-keeper.com/roleplay/10402'

In [ ]:
#df['list1_page'][1]

'<body>\n  <div class="container">\n  <div id="header-region"><div class="header-main"><div class="header-menu collapse">\n\t<div class="region" id="notify-region">\n\t\t\n\t</div>\n\t<div class="region" id="links-region"><div>\n</div></div>\n\t<div class="region" id="account-menu-region"><div>\n  <ul class="account-menu">\n    <li class="lg_hide"><a class="link" href="/search/campaigns">Browse Campaigns</a></li>\n  \t<li><a href="/login">Log In</a></li>\n    <li><a class="btn signup-btn" href="/signup">Sign Up</a></li>\n  </ul>\n\n\n</div></div>\n</div>\n<div class="header-menubar">\n\t<div class="logo"><a href="/"><img src="https://d1khas202o8dca.cloudfront.net/assets/tk_logo_light_flat-5919f3cac9928e099c11bab6f39138d5.png"></a>\n</div>\n\t\t<div class="pull-right menubar-nav">\n\t\t\t<ul class="header-links">\n\t\t\t\t<li><a class="link home-link" href="/home"><i class="icon-home"></i></a></li>\n\t\t\t\t<li><a class="link" href="/search/campaigns">Browse Campaigns</a></li>\n\t\t\t\t

# Documentação

Pela classe **char-name** conseguimos pegar o nome de todos os personagens e removê-los para facilitar o tratamento na etapa posterior.

In [ ]:
#char_names = re.findall('<div class="char-name">(.*?)</div>', df['list1_page'][0], re.DOTALL)
#set(char_names)

{'Ageric',
 'Atawulf',
 'Baldor Brandybuck',
 'Belgo',
 'Game Master',
 'Jot Walker',
 'Larsi, son of Torg'}

O dataset está separado em url da página e html da página referente a uma sessão de play by post. No html temos a seguinte estrutura para cada mensagem postada:

In [ ]:
  <div class="msg-container face front">
    <div class="message-content"> # Início do bloco pego pelo regex
      <p>Atawulf nodded accepting the parcel by...
      dice roll...
...
  <div class="msg-container face front"> # final do bloco
    <div class="message-content"> #Início do bloco seguinte
      <p><strong>Larsi, my friend,&nbsp; you can surely...

O hmtl de cada mensagem possui uma div com a classe **msg-container face front** seguida de uma div com a classe **message-content**. Dessa forma, criando um regex que pega todo conteúdo a partir do **message-content** de um balão de conversa, até o **msg-container face front** do balão seguinte, garantimos que estamos pegando todo conteúdo de cada balão.

In [ ]:
# regex que pega o bloco html equivalente a cada mensagem enviada
re.findall('<div class="message-content">(.*?)<div class="msg-container face front">', page_html, re.DOTALL)



```
# This is formatted as code
```

Com cada mensagem separada, o que nos interessa são as messagens que possuem rolagens de dados, no nosso caso só precisamos procurar por **dice roll** em cada mensagem. As mensagens que não tiverem rolagem de dados podem ser descartadas.

In [ ]:
# texto para treinamento: Atawulf nodded accepting the parcel by...
# perícia: Lore

<div class="message-content">
  <p>Atawulf nodded accepting the parcel by...</p>
    <div class="dice-roll-block">
      <a ...<i class="icon-caret-up"></i>
        dice roll 
      </a>
      <div id="roll-msg-460069" class="roll-region collapse"><div class="dice-roll-result">
        <div class="roll-value">
          <b> Lore</b>: 1D12
        </div>

**Conseguimos** separar a mensagem escrita pelo jogador inciando pela tag **p** e fechando na div com classe **dice-roll-block**. E conseguimos pegar o teste que foi feito, pegando o texto dentro da tag **b**.

In [ ]:
if 'dice roll' in match:
  train_text = strip_tags(re.findall('<p>(.*?)<div class="dice-roll-block">', match, re.DOTALL)[0]).replace('\n', '')
  skill = re.findall('<b>(.*?)</b>', match, re.DOTALL)[0].strip()

# Código

In [7]:
# Remove o nome dos personagens
def remove_char_names(text):
  char_names = set(re.findall('<div class="char-name">(.*?)</div>', text, re.DOTALL))
  
  for name in char_names: 
    text = text.replace(name, ' ') 
  
  return " ".join(text.split()) 

# Retorna o nome da skill no DnD 5e com base no dicionário
def get_5e_skill_name(old_skill_name): 
  
  # Evita um erro no regex interno do python https://stackoverflow.com/questions/3675144/regex-error-nothing-to-repeat
  old_skill_name = old_skill_name.replace('+', '').replace('*', '')
  # Evita o error: unbalanced parenthesis at position 1
  old_skill_name = old_skill_name.replace(')', ' ').replace('(', '')
  #Evita o error: error: bad escape \P at position 5
  old_skill_name = old_skill_name.replace('\\', ' ') 
  
  df_skill = df_checks.loc[df_checks['text_skill'].str.contains(old_skill_name, case=False)]
  return old_skill_name if df_skill.empty else df_skill.iloc[0]['5e_skill']

In [ ]:
import pandas as pd
df = pd.DataFrame([['apple', 'banana'], ['orange', 'pineapple']], columns=['A', 'B'])

In [ ]:
# quero saber se algum valor está contido em uma string

test = df.loc[df['B'].str.contains('Apple', case=False)]
test.iloc[0]['B']

'pineapple'

In [ ]:
#datasets_path = Path('drive/My Drive/ia/rpg/datasets/')

In [ ]:
for tk in tavern_keeper_path.iterdir():
  print(tk)

drive/My Drive/ia/rpg/tavern_keeper/taverkeeper10400-10600.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper10600-10800.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper10800-11000.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper11000-11200.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper11200-11400.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper11400-11600.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper11600-11800.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper11800-12000.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper12000-12200.csv
drive/My Drive/ia/rpg/tavern_keeper/taverkeeper12200-12400.csv


In [8]:
import re

tavern_keeper_path = Path('drive/My Drive/ia/rpg/tavern_keeper/')
skill_list = []
counter_skip = 0
conter_two_rolls = 0

# pegando o texto das mensagens onde tem as rolagens
for tk_csv_file in tavern_keeper_path.iterdir():

  df = pd.read_csv(tk_csv_file)
  # Conversão pra evitar erros de tipos de dados
  df['list1_page'] = df['list1_page'].astype(str)

  for html_page in df['list1_page']:
    # Removendo os nomes dos personagens
    html_page = remove_char_names(html_page)

    matches = re.findall('<div class="message-content">(.*?)<div class="msg-container face front">', html_page, re.DOTALL)
    for match in matches:
      if '<div class="dice-roll-block">' in match:
        
        train_text = re.findall('<p(.*?)<div class="dice-roll-block">', match, re.DOTALL)

        # Para evitar erros quando não encontra o pattern
        if not train_text:
          counter_skip += 1
          continue

        train_text = strip_tags(train_text[0]).replace('\n', '')
        skills = re.findall('<b>(.*?)</b>', match, re.DOTALL)

        # Para pegar os testes duplos. tinha um total de 615 testes duplos
        for skill in skills:
          
          # Melhor processamento pra criar um dataframe é usando dict https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe/17496530#17496530
          dict_skill = {}
          dict_skill.update({'skill': get_5e_skill_name(skill.strip()), 'backward_text': train_text, 'original_name': skill.strip()}) 
          skill_list.append(dict_skill)
        
        #print('Found %s : %s' % (get_5e_skill_name(skill), train_text) )

df_to_export = pd.DataFrame(skill_list)

In [ ]:
# Quantidade de vezes que o html não segue o padrão e o conteúdo da mensagem é ignorado
print(counter_skip)

1


In [ ]:
# Testes duplos
#print(conter_two_rolls)

615


In [9]:
df_to_export.head()

,skill,backward_text,original_name
0,History,"> nodded accepting the parcel by ’s hands, in ...",Lore
1,Performance,"> nodded accepting the parcel by ’s hands, in ...",Song
2,Arcana,">""Aye, give me but a moment to study this piec...",Craft
3,Medicine,>(Healing roll: to see if he can remember anyt...,Healing
4,Survival,>(OOC: Travel roll. Do we need even to test co...,Travel


In [10]:
df_to_export.size

18882

In [ ]:
df_to_export.sample().values[0]

array(['surname',
       '>Totally non-binding call-sign roll."Vortex Eon," hmm... ',
       'surname'], dtype=object)

In [12]:
df_to_export.loc[df_to_export['skill'] == 'History'].sample().values[0]

array(['History',
       ">Basil runs up to the hag and tears into her with his teeth, blood sprays everywhere, but she somehow manages to not fall down again.G'rrdon rushes up beside Basil and wails away with his glaive, tearing a giant chunk out of her chest.\xa0The Hag looks like shes been torn to pieces, just barely hanging on to life by just the smallest sliver of evil.OOC: The party's ability to shut her down and force her to make saves instead of attacks has been devastating. ",
       'Str'], dtype=object)

In [11]:
df_to_export.groupby('skill').size().nlargest(50)

skill
Roll               1772
Perception          226
Damage              211
Persuasion          125
dmg                  96
Nature               90
Investigation        85
Insight              79
Survival             75
Stealth              63
Athletics            62
History              53
Attack               51
Intimidation         49
Stat                 42
Initiative           35
damage               32
Arcana               29
Acrobatics           28
Strength             28
Adv                  23
Performance          23
Riddle               23
Eyes                 22
Inspire              18
Presence             18
Medicine             17
Battle               16
Con                  16
#2 atk               15
Armor                15
Claw                 15
#1 atk               14
Advantage            14
Charisma             14
Dmg                  14
DMG                  13
Wisdom               13
Attack Roll          12
Crit                 12
#3 atk               11
HP        

In [ ]:
#df_to_export.to_csv (r'drive/My Drive/ia/rpg/datasets/tk_skills.csv', index = False, header=True)

[o arquivo](https://drive.google.com/file/d/1jZ11SLh4cJrtOKqdVTG3KJBl8lxFaeaN/view?usp=sharing)

In [ ]:
import pandas as pd
new_df = pd.read_csv('drive/My Drive/ia/rpg/datasets/tk_skills.csv')

In [ ]:
new_df.groupby('skill').size().nlargest(50)

skill
Roll               1244
Perception          182
Persuasion          105
Insight              64
Survival             60
Investigation        50
Attack               42
Stealth              42
Intimidation         36
Athletics            35
History              33
Initiative           33
Eyes                 22
Performance          18
Medicine             17
Nature               17
Strength             17
Battle               15
Inspire              15
Investigate          15
#1 atk               14
Arcana               12
Charisma             12
Damage               12
HP                   11
Riddle               11
Wisdom               11
Atk                  10
Attack Roll          10
Quick (10+1=11)      10
Attack1.base          9
Feet                  9
Intelligence          9
Str                   9
Sword                 9
Acrobatics            8
Bite                  8
Craft                 8
Deception             8
Guts                  8
Mouth                 8
Power     

In [25]:
skill_list2 = []

for tk_csv_file in tavern_keeper_path.iterdir():

  df2 = pd.read_csv(tk_csv_file)
  # Conversão pra evitar erros de tipos de dados
  df2['list1_page'] = df2['list1_page'].astype(str)

  for html_page in df2['list1_page']:
    # Removendo os nomes dos personagens
    html_page = remove_char_names(html_page)

    matches = re.findall('<div class="message-content">(.*?)</div>', html_page, re.DOTALL)
    for match in matches:

      if '<div class="dice-roll-block">' in match:
        continue

      match = strip_tags(match)
      for text_skill in df_checks['text_skill']:

        if re.findall(text_skill + " (?:check|roll)", match, re.IGNORECASE):
          dict_skill = {}
          dict_skill.update({'skill': get_5e_skill_name(text_skill.strip()), 'backward_text': match, 'original_name': text_skill.strip()}) 
          skill_list2.append(dict_skill)
          #print('%s - %s' % (text_skill, match))

text_mesage_df = pd.DataFrame(skill_list2)

In [26]:
text_mesage_df.head()

,skill,backward_text,original_name
0,Survival,the boy whines but we collectively pull him ...,Travel
1,Perception,"Keridan continues threw the woods,the woods s...",Perception
2,Perception,darkness hits and the area is so dark it’s h...,Perception
3,History,"Ibel smiles at Sangra's eagerness, and pats h...",Lore
4,Perception,"Hearing grunt in pain, charged through the mi...",Awareness


In [27]:
text_mesage_df.size

174

In [ ]:
text_mesage_df.loc[text_mesage_df['skill'] == 'Perception'].sample().values[0]

array(['Perception',
       ' Luckily, Typhoon and Pious found a path through the larger rocks that allowed them to move at a quick speed. They were able to cover a bit more ground than the Hub as the rig moved forward.\xa0As you crest the Pawn, you see a large fissure that has opened up in the ground, and leading to a large cavern below. It looks like a pretty sizable drop, too. Not something a vehicle could get down in safely.Typhoon and Pious, please each make a Notice roll, difficulty +6From on-board the Hub, the three of you see something different, coming from a ways off towards your position. The large-wheeled, driller of\xa0NDevor company. ',
       'Notice'], dtype=object)

In [ ]:
text_mesage_df.groupby('skill').size().nlargest(50)

skill
Perception       24
Survival          9
Deception         4
History           4
Investigation     4
Persuasion        3
Arcana            2
Medicine          2
Athletics         1
Insight           1
Intimidation      1
Performance       1
Religion          1
Stealth           1
dtype: int64

In [ ]:
text_mesage_df.groupby('original_name').size().nlargest(50)

original_name
Perception       13
Travel            7
Awareness         6
Lore              4
Arcana            2
Investigate       2
Listen            2
Cunning           2
Courtesy          2
Corruption        2
Explore           1
Persuasion        1
Awe               1
Stealth           1
Spot              1
Song              1
Search            1
Religion          1
Notice            1
Healing           1
Medicine          1
Athletics         1
Investigation     1
Insight           1
Hunting           1
Vigilant          1
dtype: int64

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Fazer o wordcloud com base no df_estrat ao invés de df

def print_wordcloud(skill_name):
  skills_backward = text_mesage_df[text_mesage_df['skill'] == skill_name]
  document = ' '.join(skills_backward['backward_text'])

  wordcloud = WordCloud(background_color="white").generate(document)
  plt.figure(figsize=(10,10))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  print(skill_name)

In [ ]:
df_to_export.groupby('original_name').size().nlargest(50)

original_name
Roll             1772
Damage            211
dmg                96
Perception         90
Insight            67
Attack             51
Stealth            47
Stat               42
Wild               42
wild               39
Persuade           38
Initiative         35
Courtesy           34
Investigation      33
Athletics          32
Awareness          32
damage             32
Awe                31
Lore               30
awareness          29
Strength           28
Persuasion         26
Travel             24
Adv                23
Riddle             23
perception         23
Eyes               22
Song               20
Inspire            18
Presence           18
Battle             16
Con                16
Hunting            16
Investigate        16
#2 atk             15
Armor              15
Claw               15
#1 atk             14
Advantage          14
Charisma           14
Dmg                14
                   13
DMG                13
Notice             13
Survival          

In [31]:
df_to_export.append(text_mesage_df, ignore_index=False)

,skill,backward_text,original_name
0,History,"> nodded accepting the parcel by ’s hands, in ...",Lore
1,Performance,"> nodded accepting the parcel by ’s hands, in ...",Song
2,Arcana,">""Aye, give me but a moment to study this piec...",Craft
3,Medicine,>(Healing roll: to see if he can remember anyt...,Healing
4,Survival,>(OOC: Travel roll. Do we need even to test co...,Travel
...,...,...,...
53,Medicine,I almost forgot; Banog’s horse was hurt in th...,Healing
54,Perception,"""I get bored here and we need new blood from ...",Perception
55,Religion,Ismark standing so the priest can't see gestu...,Religion
56,Perception,Amarla is waiting for Qanhaz.OOC: Perception ...,Perception


In [30]:
df_to_export.shape

(6294, 3)

In [32]:
df_to_export.to_csv (r'drive/My Drive/ia/rpg/datasets/tk_skills2.csv', index = False, header=True)